# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-05-02, 12:51:07

   -> Summany DataFrame : 123 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250430,DB1R000100,Prepay Revenue,2025-04-30,20250421,20250430,"574,374,616","1,148,749,232","1,147,773,342","1,147,773,342",0,"12,427,045",4,170,1850
1,20250430,DB2R000100,Postpaid Revenue,2025-04-30,20250421,20250430,"953,433,729","1,906,867,458","1,905,233,358","1,905,233,358",0,"33,543,193",4,170,1840
2,20250430,DB2R010100,Postpaid Revenue B2C,2025-04-30,20250421,20250430,"929,431,823","1,858,863,647","1,857,232,655","1,857,232,655",0,"33,452,551",4,170,1836
3,20250430,DB2R020100,Postpaid Revenue B2B,2025-04-30,20250421,20250430,"24,001,906","48,003,812","48,000,702","48,000,702",0,"90,642",4,170,1830


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250430,DB1S000500,Prepay Usage Subs,2025-04-30,20250421,20250430,"112,972,069","225,944,108","225,766,948","225,766,948",0,"2,521,407",4,170,1850
1,20250430,DB2S000500,Postpaid Active Subs,2025-04-30,20250421,20250430,"57,980,756","115,961,494","115,856,001","115,856,001",0,"1,930,142",4,170,1848
2,20250430,DB2S010500,Postpaid Active Subs B2C,2025-04-30,20250421,20250430,"51,096,068","102,192,118","102,108,129","102,108,129",0,"1,737,083",4,170,1848
3,20250430,DB2S020400,Postpaid Active Sub B2B,2025-04-30,20250421,20250430,"6,884,688","13,769,376","13,747,872","13,747,872",0,"193,059",4,170,1830


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-05-02, 12:51:09

   -> Daily DataFrame : 80 rows, 16 columns

TDMDBPR : Disconnected


In [12]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250430,DB1R000100,Prepay Revenue,20250421,2025-04-30,"55,363,443","110,726,885","110,635,130","110,635,130",0,"1,161,318",0,0,4,170,185
1,20250430,DB1R000100,Prepay Revenue,20250422,2025-04-30,"52,837,131","105,674,262","105,582,950","105,582,950",0,"1,140,129",0,0,4,170,185
2,20250430,DB1R000100,Prepay Revenue,20250423,2025-04-30,"54,567,466","109,134,932","109,044,853","109,044,853",0,"1,191,479",0,0,4,170,185
3,20250430,DB1R000100,Prepay Revenue,20250424,2025-04-30,"55,437,593","110,875,187","110,780,310","110,780,310",0,"1,254,309",0,0,4,170,185
4,20250430,DB1R000100,Prepay Revenue,20250425,2025-04-30,"58,889,190","117,778,381","117,681,118","117,681,118",0,"1,222,052",0,0,4,170,185
5,20250430,DB1R000100,Prepay Revenue,20250426,2025-04-30,"56,535,262","113,070,525","112,983,499","112,983,499",0,"1,253,942",0,0,4,170,185
6,20250430,DB1R000100,Prepay Revenue,20250427,2025-04-30,"55,657,201","111,314,402","111,220,250","111,220,250",0,"1,205,802",0,0,4,170,185
7,20250430,DB1R000100,Prepay Revenue,20250428,2025-04-30,"56,448,873","112,897,745","112,799,926","112,799,926",0,"1,211,479",0,0,4,170,185
8,20250430,DB1R000100,Prepay Revenue,20250429,2025-04-30,"56,736,318","113,472,635","113,371,168","113,371,168",0,"1,216,212",0,0,4,170,185
9,20250430,DB1R000100,Prepay Revenue,20250430,2025-04-30,"71,902,139","143,804,278","143,674,137","143,674,137",0,"1,570,324",0,0,4,170,185


In [7]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250430,DB2R000100,Postpaid Revenue,20250421,2025-04-30,"81,661,596","163,323,192","163,187,842","163,187,842",0,"2,854,194",0,0,4,170,183
1,20250430,DB2R000100,Postpaid Revenue,20250422,2025-04-30,"81,556,554","163,113,108","162,983,860","162,983,860",0,"2,843,665",0,0,4,170,185
2,20250430,DB2R000100,Postpaid Revenue,20250423,2025-04-30,"80,212,666","160,425,332","160,280,993","160,280,993",0,"2,781,445",0,0,4,170,185
3,20250430,DB2R000100,Postpaid Revenue,20250424,2025-04-30,"80,203,016","160,406,031","160,259,020","160,259,020",0,"2,831,305",0,0,4,170,183
4,20250430,DB2R000100,Postpaid Revenue,20250425,2025-04-30,"81,373,019","162,746,037","162,605,270","162,605,270",0,"2,845,510",0,0,4,170,183
5,20250430,DB2R000100,Postpaid Revenue,20250426,2025-04-30,"89,110,607","178,221,215","178,054,508","178,054,508",0,"3,279,051",0,0,4,170,185
6,20250430,DB2R000100,Postpaid Revenue,20250427,2025-04-30,"88,788,004","177,576,008","177,427,331","177,427,331",0,"3,238,734",0,0,4,170,183
7,20250430,DB2R000100,Postpaid Revenue,20250428,2025-04-30,"93,788,045","187,576,090","187,425,009","187,425,009",0,"3,170,472",0,0,4,170,185
8,20250430,DB2R000100,Postpaid Revenue,20250429,2025-04-30,"268,364,604","536,729,208","536,275,116","536,275,116",0,"9,429,182",0,0,4,170,185
9,20250430,DB2R000100,Postpaid Revenue,20250430,2025-04-30,"8,375,619","16,751,237","16,734,409","16,734,409",0,"269,635",0,0,4,170,183


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250430,DB2R010100,Postpaid Revenue B2C,20250421,2025-04-30,"79,589,469","159,178,938","159,044,563","159,044,563",0,"2,843,357",0,0,4,170,183
1,20250430,DB2R010100,Postpaid Revenue B2C,20250422,2025-04-30,"79,387,713","158,775,426","158,646,444","158,646,444",0,"2,832,696",0,0,4,170,185
2,20250430,DB2R010100,Postpaid Revenue B2C,20250423,2025-04-30,"78,644,209","157,288,418","157,144,231","157,144,231",0,"2,772,553",0,0,4,170,185
3,20250430,DB2R010100,Postpaid Revenue B2C,20250424,2025-04-30,"78,815,628","157,631,256","157,484,530","157,484,530",0,"2,820,765",0,0,4,170,183
4,20250430,DB2R010100,Postpaid Revenue B2C,20250425,2025-04-30,"79,620,671","159,241,341","159,101,023","159,101,023",0,"2,838,723",0,0,4,170,183
5,20250430,DB2R010100,Postpaid Revenue B2C,20250426,2025-04-30,"88,125,095","176,250,191","176,083,682","176,083,682",0,"3,268,376",0,0,4,170,185
6,20250430,DB2R010100,Postpaid Revenue B2C,20250427,2025-04-30,"87,907,473","175,814,946","175,666,526","175,666,526",0,"3,235,334",0,0,4,170,183
7,20250430,DB2R010100,Postpaid Revenue B2C,20250428,2025-04-30,"86,398,818","172,797,635","172,646,588","172,646,588",0,"3,165,008",0,0,4,170,185
8,20250430,DB2R010100,Postpaid Revenue B2C,20250429,2025-04-30,"264,996,481","529,992,962","529,539,156","529,539,156",0,"9,420,659",0,0,4,170,185
9,20250430,DB2R010100,Postpaid Revenue B2C,20250430,2025-04-30,"5,946,267","11,892,535","11,875,913","11,875,913",0,"255,080",0,0,4,169,179


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250401


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250430,DB2R020100,Postpaid Revenue B2B,20250421,2025-04-30,"2,072,127","4,144,254","4,143,279","4,143,279",0,"10,837",0,0,4,170,183
1,20250430,DB2R020100,Postpaid Revenue B2B,20250422,2025-04-30,"2,168,841","4,337,682","4,337,417","4,337,417",0,"10,969",0,0,4,170,183
2,20250430,DB2R020100,Postpaid Revenue B2B,20250423,2025-04-30,"1,568,457","3,136,914","3,136,762","3,136,762",0,"8,892",0,0,4,170,183
3,20250430,DB2R020100,Postpaid Revenue B2B,20250424,2025-04-30,"1,387,387","2,774,775","2,774,489","2,774,489",0,"10,540",0,0,4,170,183
4,20250430,DB2R020100,Postpaid Revenue B2B,20250425,2025-04-30,"1,752,348","3,504,696","3,504,247","3,504,247",0,"6,786",0,0,4,170,183
5,20250430,DB2R020100,Postpaid Revenue B2B,20250426,2025-04-30,"985,512","1,971,024","1,970,826","1,970,826",0,"10,675",0,0,4,170,183
6,20250430,DB2R020100,Postpaid Revenue B2B,20250427,2025-04-30,"880,531","1,761,063","1,760,806","1,760,806",0,"3,400",0,0,4,170,183
7,20250430,DB2R020100,Postpaid Revenue B2B,20250428,2025-04-30,"7,389,228","14,778,455","14,778,421","14,778,421",0,"5,464",0,0,4,170,183
8,20250430,DB2R020100,Postpaid Revenue B2B,20250429,2025-04-30,"3,368,123","6,736,246","6,735,960","6,735,960",0,"8,523",0,0,4,170,183
9,20250430,DB2R020100,Postpaid Revenue B2B,20250430,2025-04-30,"2,429,351","4,858,703","4,858,496","4,858,496",0,"14,555",0,0,4,170,183


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]